In [238]:
# Import Required Packages
import lxml
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib
import urllib.request as urllib2
import os
import requests
import pandas as pd

In [239]:
# Get URL
url = 'https://pittsburghpanthers.com/roster.aspx?path=softball'
#url = 'https://hokiesports.com/roster.aspx?roster=279&path=softball'
#url = 'https://goduke.com/sports/softball/roster'
# establish a session to request website
user_agent = {'User-agent': 'Mozilla/5.0'}
urlSplit = url.split('/')
# Create string to represent the base url
baseUrl = urlSplit[0] + '/' + urlSplit[1] + '/' + urlSplit[2]

In [240]:
# Check if we get a response
r = requests.get(url, headers = user_agent)
# Make the soup
soup = BeautifulSoup(r.text, 'html.parser')

In [241]:
# Create a list of the players on the roster.
players = list()
firstName = list()
lastName = list()
numbers = list()
bat = list()
throw = list()
pos = list()

# Scrape the players name
for item in soup.findAll('td', {"class": "sidearm-table-player-name"}):
    name = item.text
    players.append(item.text)
    if('  ' in name):
        fullName = name.split('  ')
    else: fullName = name.split(' ')
    firstName.append(fullName[0])
    lastName.append(fullName[1])
players

# Scrape te players jersey number
for item in soup.findAll('td', {"class": "roster_jerseynum"}):
    numbers.append(item.text)
    
    
# Scrape the players batting/throwing side    
for item in soup.findAll('td', {"class": 'sidearm-table-custom-field'}):
    b_t = item.text.split('/')
    if(b_t[0] == 'R'):
        bat.append('Right')
    else: bat.append('Left')
    if(b_t[1] == 'R'):
        throw.append('Right')
    else: throw.append('Left')
        
# Scrape the players position
for item in soup.findAll('td', {"class": "rp_position_short"}):
    pos.append(item.text)
    
    
#########################################################
# Get user input for Team Abbrieviation:
#########################################################
ta = 'PIT'

#########################################################
# Get user input for Team Organization:
#########################################################
to = 'Pittsburg'

In [242]:
# Construct the Data Frame and Display
frame = pd.DataFrame(numbers, columns=['Jersey'])
frame['First Name'] = firstName
frame['Last Name'] = lastName
frame['Pitching Strength'] = throw
frame['Hitting Strength'] = bat
frame['Position'] = pos
frame['Team Abbrieviation'] = ta
frame['Team Organization'] = to
frame

,Jersey,First Name,Last Name,Pitching Strength,Hitting Strength,Position,Team Abbrieviation,Team Organization
0,1,Connor,McGaffic,Right,Right,OF,PIT,Pittsburg
1,2,Jada,Crittendon,Right,Right,OF,PIT,Pittsburg
2,3,Hannah,Bach,Right,Right,C/U,PIT,Pittsburg
3,4,Makenzie,Stiles,Right,Right,P,PIT,Pittsburg
4,5,Hunter,Levesque,Right,Right,OF/INF,PIT,Pittsburg
5,6,Hope,Alley,Right,Right,INF,PIT,Pittsburg
6,7,Kaitlyn,Kruger,Right,Right,INF/OF,PIT,Pittsburg
7,8,Kayla,Lane,Right,Right,INF,PIT,Pittsburg
8,9,Chandler,Walter,Right,Right,INF,PIT,Pittsburg
9,11,LoLo,Sanchez,Right,Left,INF,PIT,Pittsburg


In [243]:
# create a list of links to player pages and display
playerAnchor = list()
anchors = soup.select('td.sidearm-table-player-name a', href = True)
for a in anchors:
    playerAnchor.append(baseUrl + a['href'])
playerAnchor

['https://pittsburghpanthers.com/roster.aspx?rp_id=10506',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10574',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10497',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10511',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10505',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10510',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10504',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10512',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10509',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10572',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10513',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10502',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10498',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10508',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10500',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10499',
 'https://pittsburghpanthers.com/roster.aspx?rp_id=10507

In [252]:
#########################################################
# User defines the path of where they save the file in os.path.basename
#########################################################


# find players and download images.
for idx, playerLink in enumerate(playerAnchor):
    # reference players link and create soup
    r = requests.get(playerLink, headers = user_agent)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # find images of the players
    player = soup.find('img', {'alt': players[idx]})
    print(player)
    # download the image
    if(player != None):
        imgurl = baseUrl + player['src'].split('?')[0]
        urllib2.urlretrieve(imgurl, os.path.join('./pitt/', os.path.basename(imgurl)))

<img alt="Connor McGaffic" src="/images/2019/10/15/1_McGaffic.jpg?width=300"/>
<img alt="Jada Crittendon" src="/images/2019/10/15/2_Crittendon.jpg?width=300"/>
None
<img alt="Makenzie Stiles" src="/images/2019/10/15/4_Stiles.jpg?width=300"/>
<img alt="Hunter Levesque" src="/images/2019/10/15/5_Levesque.jpg?width=300"/>
<img alt="Hope Alley" src="/images/2019/10/15/6_Alley.jpg?width=300"/>
<img alt="Kaitlyn Kruger" src="/images/2019/10/15/7_Kruger.jpg?width=300"/>
<img alt="Kayla Lane" src="/images/2019/10/15/8_Lane.jpg?width=300"/>
<img alt="Chandler Walter" src="/images/2019/10/15/9_Walter.jpg?width=300"/>
<img alt="LoLo Sanchez" src="/images/2019/10/15/11_Sanchez.jpg?width=300"/>
<img alt="Taegen Lewellen" src="/images/2019/10/15/12_Lewellen.jpg?width=300"/>
<img alt="Brie Horrell" src="/images/2019/10/15/14_Horrell.jpg?width=300"/>
<img alt="Walker Barbee" src="/images/2019/10/15/18_Barbee.jpg?width=300"/>
None
None
<img alt="Morgan Batesole" src="/images/2019/10/15/26_Batesole.jpg?